In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'sklearn'

In [182]:
cwd = os.path.join(os.getcwd(), 'Group Coursework Brief-20221106', 'Data_Files', 'Data_Files')
dirName_trainData = os.path.join(cwd, 'epl-training.csv')

In [183]:
df_epl_train = pd.read_csv(dirName_trainData)

In [184]:
df_epl_train.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,19/08/00,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,Rob Harris,...,14.0,4.0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0
1,19/08/00,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,Graham Barber,...,10.0,5.0,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0
2,19/08/00,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,Barry Knight,...,3.0,9.0,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0
3,19/08/00,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,Andy D'Urso,...,4.0,6.0,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0
4,19/08/00,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,Dermot Gallagher,...,8.0,6.0,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0


In [185]:
# Transform the date column from strings into datetime objects
df_epl_train["Date"] = pd.to_datetime(df_epl_train["Date"], dayfirst=True)

In [186]:
df_epl_train.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,2000-08-19,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,Rob Harris,...,14.0,4.0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0
1,2000-08-19,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,Graham Barber,...,10.0,5.0,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0
2,2000-08-19,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,Barry Knight,...,3.0,9.0,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0
3,2000-08-19,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,Andy D'Urso,...,4.0,6.0,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0
4,2000-08-19,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,Dermot Gallagher,...,8.0,6.0,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0


AIM: A function that takes as input the date, HomeTeam and AwayTeam. It will filter the df_epl_train dataframe for matches between HomeTeam and AwayTeam that took place before the input date. Then take an average of the 
columns like HR, AR, etc. This will provide us with the past stats for games played in past between the two teams. We can then use these past stats (between the two teams) as features to input into the classifier.

First, filter the dataframe to include only matches where date is less than date specified and also only include matches where HomeTeam=input(HomeTeam) and AwayTeam=input(AwayTeam):

In [187]:
# This function will take as input a date, HomeTeam and AwayTeam and output a filtered dataframe where the matches shown are played before input data and match is between HomeTeam and AwayTeam

# For Example:
# date = "24/06/2020"
# HomeTeam = "Newcastle"
# AwayTeam = "Aston Villa"

def filter_dataframe(date, HomeTeam, AwayTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && HomeTeam=input(HomeTeam) && AwayTeam=input(AwayTeam)
    df_epl_train_filtered = df_epl_train.copy()
    df_epl_train_filtered = df_epl_train_filtered[(df_epl_train.Date<date) & (df_epl_train.HomeTeam==HomeTeam) & (df_epl_train.AwayTeam==AwayTeam)]

    # Return filtered dataframe
    return df_epl_train_filtered

# An example to see what the function does:
print(filter_dataframe("24/06/2020", "Newcastle", "Aston Villa"))

           Date   HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
377  2001-05-19  Newcastle  Aston Villa   3.0   0.0   H   2.0   0.0   H   
486  2001-11-03  Newcastle  Aston Villa   3.0   0.0   H   1.0   0.0   H   
1105 2003-04-21  Newcastle  Aston Villa   1.0   1.0   D   1.0   0.0   H   
1244 2003-11-01  Newcastle  Aston Villa   1.0   1.0   D   1.0   1.0   D   
1825 2005-04-02  Newcastle  Aston Villa   0.0   3.0   A   0.0   1.0   A   
2041 2005-12-03  Newcastle  Aston Villa   1.0   1.0   D   1.0   0.0   H   
2526 2007-01-31  Newcastle  Aston Villa   3.0   1.0   H   2.0   1.0   H   
2679 2007-08-18  Newcastle  Aston Villa   0.0   0.0   D   0.0   0.0   D   
3148 2008-11-03  Newcastle  Aston Villa   2.0   0.0   H   0.0   0.0   D   
3818 2010-08-22  Newcastle  Aston Villa   6.0   0.0   H   3.0   0.0   H   
4418 2012-02-05  Newcastle  Aston Villa   2.0   1.0   H   1.0   1.0   D   
4587 2012-09-02  Newcastle  Aston Villa   1.0   1.0   D   0.0   1.0   A   
5206 2014-02-23  Newcastl

We now find the average of each of the columns that we need from this filtered dataframe e.g. HST, AST:

In [188]:
# This function takes as input the filtered dataframe from previous cell, features to average and a dictionary,
# it then appends an average of each feature to the dictionary

def average_columns(features, avg_features, filtered_df):
    for feature in features:
        df_col_means = df_epl_train_filtered[feature].mean()
        avg_features[feature].append(df_col_means)

We now run the two functions on each row of the original dataframe to fill the dictionary with averages for each row

In [189]:
# Run the two functions for each row of our df_epl_train dataframe to fill dictionary with AVG for each match
# NOTE: Some matches won't have past stats since the two teams may not have played against each other in past or we might not have the data

# These are the features we want to get averages for
features = ["FTHG","FTAG","HTHG","HTAG","HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]
avg_features = {
                    "FTHG": [],
                    "FTAG": [],
                    "HTHG": [],
                    "HTAG": [],
                    "HS"  : [],
                    "AS"  : [],
                    "HST" : [],
                    "AST" : [],
                    "HF"  : [],
                    "AF"  : [],
                    "HC"  : [],
                    "AC"  : [],
                    "HY"  : [],
                    "AY"  : [],
                    "HR"  : [],
                    "AR"  : []
                }

# Run the two functions on each row of the df_epl_train and fill the dictionary
# For each row in the dataframe
for index, row in df_epl_train.iterrows():
    # Filter the dataframe to only show matches played between those teams and before the certain date
    df_epl_train_filtered = filter_dataframe(row["Date"],row["HomeTeam"],row["AwayTeam"])
    # Get averages from the filtered dataframe and add the the dictionary
    average_columns(features, avg_features, df_epl_train_filtered)

Add these average feature lists in the dictionary back into the original dataframe as columns:

In [190]:
# Add a column for each of these feature averages using our list of values from the dictionary
df_epl_train_updated = df_epl_train.copy()
features = ["FTHG","FTAG","HTHG","HTAG","HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]

for feature in features:
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_AVG"] = feature_vals

This is the new dataframe with the added columns with past average statistics for each row:

In [191]:
# Now this dataframe contains our original data + the average of the past stats for each row
df_epl_train_updated

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST_AVG,AST_AVG,HF_AVG,AF_AVG,HC_AVG,AC_AVG,HY_AVG,AY_AVG,HR_AVG,AR_AVG
0,2000-08-19,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,Rob Harris,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-08-19,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,Graham Barber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-08-19,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,Barry Knight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-08-19,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,Andy D'Urso,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-08-19,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,Dermot Gallagher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8481,2022-10-29,Newcastle,Aston Villa,4.0,0.0,H,1.0,0.0,H,P Tierney,...,5.888889,4.500000,12.000000,14.388889,6.944444,4.888889,1.388889,2.666667,0.222222,0.111111
8482,2022-10-29,Fulham,Everton,0.0,0.0,D,0.0,0.0,D,J Brooks,...,5.400000,5.933333,13.333333,12.333333,4.533333,5.266667,1.333333,1.133333,0.133333,0.133333
8483,2022-10-29,Liverpool,Leeds,1.0,2.0,A,1.0,1.0,D,M Oliver,...,8.000000,3.666667,11.166667,12.833333,6.666667,2.500000,1.166667,1.500000,0.166667,0.000000
8484,2022-10-30,Arsenal,Nott'm Forest,5.0,0.0,H,1.0,0.0,H,S Hooper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NOTE: Some rows have nan values, these are matches where teams might not have played each other before (or we dont have the past match data for them)

So we need to remove these rows (from the dataframe) where there are nan values. This is required to run the classifier training

In [192]:
# NOTE: We must remove the rows in the dataframe where the average values of stats/features are 'nan';
# we get these values because either the two teams have not played a match in the past OR because we have
# not got the past stats for these matches. We cannot use the 'nan' values for the classifier training and 
# hence have to remove these rows. We can then train a classifier using this final dataframe.

# In the final model/classifier, in the case where we DO NOT have these past stats of the two teams playing, 
# we need to switch back to using the OLD classifier which only took the 4 basic fetaures: day, month, 
# HomeTeam and AwayTeam.

# In the case where we DO have these past stats for two teams playing each other, we can use this model/classifier 
# and input the features like HST_AVG and AST_AVG. We would find these by using the filter_dataframe() and 
# average_columns() functions to find them for any two specific teams playing each other on some date.

# Remove any rows with nan
df_epl_train_final = df_epl_train_updated.dropna()
df_epl_train_final

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST_AVG,AST_AVG,HF_AVG,AF_AVG,HC_AVG,AC_AVG,HY_AVG,AY_AVG,HR_AVG,AR_AVG
380,2001-08-18,Charlton,Everton,1.0,2.0,A,0.0,0.0,D,N. S. Barry,...,7.000000,4.000000,15.000000,21.000000,13.000000,6.000000,1.000000,4.000000,0.000000,0.000000
382,2001-08-18,Leeds,Southampton,2.0,0.0,H,0.0,0.0,D,C. R. Wilkes,...,7.000000,2.000000,10.000000,11.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000
384,2001-08-18,Liverpool,West Ham,2.0,1.0,H,1.0,1.0,D,J. T. Winter,...,10.000000,3.000000,9.000000,6.000000,5.000000,3.000000,0.000000,1.000000,0.000000,1.000000
385,2001-08-18,Middlesbrough,Arsenal,0.0,4.0,A,0.0,1.0,A,G. P. Barber,...,3.000000,8.000000,10.000000,11.000000,5.000000,8.000000,1.000000,1.000000,1.000000,0.000000
386,2001-08-18,Sunderland,Ipswich,1.0,0.0,H,1.0,0.0,H,G. Poll,...,9.000000,3.000000,15.000000,10.000000,4.000000,3.000000,1.000000,5.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8480,2022-10-29,Crystal Palace,Southampton,1.0,0.0,H,1.0,0.0,H,M Salisbury,...,3.800000,4.300000,13.400000,14.100000,5.600000,4.900000,1.900000,1.600000,0.100000,0.100000
8481,2022-10-29,Newcastle,Aston Villa,4.0,0.0,H,1.0,0.0,H,P Tierney,...,5.888889,4.500000,12.000000,14.388889,6.944444,4.888889,1.388889,2.666667,0.222222,0.111111
8482,2022-10-29,Fulham,Everton,0.0,0.0,D,0.0,0.0,D,J Brooks,...,5.400000,5.933333,13.333333,12.333333,4.533333,5.266667,1.333333,1.133333,0.133333,0.133333
8483,2022-10-29,Liverpool,Leeds,1.0,2.0,A,1.0,1.0,D,M Oliver,...,8.000000,3.666667,11.166667,12.833333,6.666667,2.500000,1.166667,1.500000,0.166667,0.000000


In [193]:
# Turn the catergorical data into labels using same method from before
df_epl_train_final["AwayTeam_Enc"] = df_epl_train_final["AwayTeam"].astype("category").cat.codes
df_epl_train_final["HomeTeam_Enc"] = df_epl_train_final["HomeTeam"].astype("category").cat.codes

# Transform the date column into day and month columns and Add into dataframe (Extract days & months from date)
df_epl_train_final["Date"] = pd.to_datetime(df_epl_train_final["Date"])
df_epl_train_final["Day"] = df_epl_train_final["Date"].dt.day
df_epl_train_final["Month"] = df_epl_train_final["Date"].dt.month 
df_epl_train_final["Year"] = df_epl_train_final["Date"].dt.year

# Check the final updated dataframe
df_epl_train_final

/var/folders/qp/cqr59m4j0g90ckphpxxl1_c80000gn/T/ipykernel_1127/3127087743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epl_train_final["AwayTeam_Enc"] = df_epl_train_final["AwayTeam"].astype("category").cat.codes
/var/folders/qp/cqr59m4j0g90ckphpxxl1_c80000gn/T/ipykernel_1127/3127087743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epl_train_final["HomeTeam_Enc"] = df_epl_train_final["HomeTeam"].astype("category").cat.codes
/var/folders/qp/cqr59m4j0g90ckphpxxl1_c80000gn/T/ipykernel_1127/

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AC_AVG,HY_AVG,AY_AVG,HR_AVG,AR_AVG,AwayTeam_Enc,HomeTeam_Enc,Day,Month,Year
380,2001-08-18,Charlton,Everton,1.0,2.0,A,0.0,0.0,D,N. S. Barry,...,6.000000,1.000000,4.000000,0.000000,0.000000,14,10,18,8,2001
382,2001-08-18,Leeds,Southampton,2.0,0.0,H,0.0,0.0,D,C. R. Wilkes,...,1.000000,0.000000,0.000000,0.000000,0.000000,31,19,18,8,2001
384,2001-08-18,Liverpool,West Ham,2.0,1.0,H,1.0,1.0,D,J. T. Winter,...,3.000000,0.000000,1.000000,0.000000,1.000000,38,21,18,8,2001
385,2001-08-18,Middlesbrough,Arsenal,0.0,4.0,A,0.0,1.0,A,G. P. Barber,...,8.000000,1.000000,1.000000,1.000000,0.000000,0,24,18,8,2001
386,2001-08-18,Sunderland,Ipswich,1.0,0.0,H,1.0,0.0,H,G. Poll,...,3.000000,1.000000,5.000000,0.000000,0.000000,18,33,18,8,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8480,2022-10-29,Crystal Palace,Southampton,1.0,0.0,H,1.0,0.0,H,M Salisbury,...,4.900000,1.900000,1.600000,0.100000,0.100000,31,12,29,10,2022
8481,2022-10-29,Newcastle,Aston Villa,4.0,0.0,H,1.0,0.0,H,P Tierney,...,4.888889,1.388889,2.666667,0.222222,0.111111,1,25,29,10,2022
8482,2022-10-29,Fulham,Everton,0.0,0.0,D,0.0,0.0,D,J Brooks,...,5.266667,1.333333,1.133333,0.133333,0.133333,14,15,29,10,2022
8483,2022-10-29,Liverpool,Leeds,1.0,2.0,A,1.0,1.0,D,M Oliver,...,2.500000,1.166667,1.500000,0.166667,0.000000,19,21,29,10,2022


Create our X and y matrix and split into a training and test set:

In [194]:
# Create the input features matrix X (made of day, month, HomeTeam, AwayTeam, FTHG_AVG, FTAG_AVG, etc)
# Create the output values y vector (made of FTR)
# Take these values from the transformed dataframe

X = df_epl_train_final.loc[:,['Day', 'Month', 'HomeTeam_Enc', 'AwayTeam_Enc','FTHG_AVG','FTAG_AVG','HTHG_AVG','HTAG_AVG','HS_AVG','AS_AVG','HST_AVG','AST_AVG','HF_AVG','AF_AVG','HC_AVG','AC_AVG','HY_AVG','AY_AVG','HR_AVG','AR_AVG']].values
y = df_epl_train_final.loc[:,'FTR'].values

# Split the training data in a 80-20 split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=22)

# Encode the y output values as well
FTR_encoder = LabelEncoder()
y_train = FTR_encoder.fit_transform(y_train);

Now we can test using the different classifiers:

In [195]:
# Create an empty Tree model
DT_Model = DecisionTreeClassifier(random_state=42)
# Fit the model using training data
DT_Model.fit(X_train, y_train)
# Make predictions using the model we have created
DT_predictions_test = DT_Model.predict(X_test)
# Reconverting prediction values (i.e. 0, 1 or 2) back into (H, D or A) using the FTR_encoder defined in earlier cell
DT_predictions_test = FTR_encoder.inverse_transform(DT_predictions_test)

In [196]:
print(accuracy_score(DT_predictions_test, y_test))
print(classification_report(DT_predictions_test, y_test))

0.4015748031496063
              precision    recall  f1-score   support

           A       0.37      0.35      0.36       404
           D       0.27      0.26      0.27       367
           H       0.49      0.52      0.50       626

    accuracy                           0.40      1397
   macro avg       0.38      0.38      0.38      1397
weighted avg       0.40      0.40      0.40      1397



In [197]:
# Create an empty KNN model
KNN_Model = KNeighborsClassifier(n_neighbors=6)
# Fit the model using training data
KNN_Model.fit(X_train, y_train)
# Make predictions using the model we have created
KNN_predictions_test = KNN_Model.predict(X_test)
KNN_predictions_test = FTR_encoder.inverse_transform(KNN_predictions_test)

In [198]:
print(accuracy_score(KNN_predictions_test, y_test))
print(classification_report(KNN_predictions_test, y_test))

0.4352183249821045
              precision    recall  f1-score   support

           A       0.49      0.35      0.41       544
           D       0.15      0.28      0.19       184
           H       0.55      0.55      0.55       669

    accuracy                           0.44      1397
   macro avg       0.40      0.39      0.38      1397
weighted avg       0.48      0.44      0.45      1397



In [199]:
# Create an empty Random Forest model
RF_Model = RandomForestClassifier(n_estimators=50, random_state=42)
# Fit the model using training data
RF_Model.fit(X_train, y_train)
# Make predictions using the model we have created
RF_predictions_test = RF_Model.predict(X_test)
RF_predictions_test = FTR_encoder.inverse_transform(RF_predictions_test)

In [200]:
print(accuracy_score(RF_predictions_test, y_test))
print(classification_report(RF_predictions_test, y_test))

0.506084466714388
              precision    recall  f1-score   support

           A       0.47      0.44      0.46       411
           D       0.10      0.27      0.15       131
           H       0.74      0.57      0.65       855

    accuracy                           0.51      1397
   macro avg       0.44      0.43      0.42      1397
weighted avg       0.60      0.51      0.54      1397

